In [ ]:
!pip install python-terrier

In [ ]:
import pandas as pd
import pyterrier as pt
import os

In [ ]:
if not pt.started():
    pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
import csv
import pandas as pd
import numpy as np
import string
import random

In [ ]:
recipe_dataset = pd.read_csv('/content/recipes.csv')

title_list = []
for i in range(len(r_df)):
  title_list.append([str(r_df["title"][i]),str(r_df["docno"][i]),str(r_df["rating"][i]),str(r_df["ingredients"][i]),str(r_df["instructions"][i])])
re_df = pd.DataFrame(title_list, columns =["title","docno","rating","ingredients","instructions"])

In [ ]:
recipe_indexer2 = pt.DFIndexer("./b")
indexref2 = recipe_indexer2.index(re_df["title"],re_df)
recipe_index2 = pt.IndexFactory.of(indexref2)
print(recipe_index2.getCollectionStatistics().toString())

05:05:19.212 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (vQ0ALYW) - further warnings are suppressed
05:05:25.382 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
Number of documents: 46470
Number of terms: 7971
Number of postings: 167494
Number of fields: 0
Number of tokens: 168132
Field names: []
Positions:   false



In [ ]:
jg_queries = pd.DataFrame([
    ["jg_q01", "salami pasta"],
    ["jg_q02", "fruit pizza"],
    ["jg_q03", "pineapple and strawberry"],
    ["jg_q04", "masala chicken"],
    ["jg_q05", "beef steak"],
    ["jg_q06", "french fries"],
    ["jg_q07", "tuna sushi"],
    ["jg_q08", "strawberry lemonade"],
    ["jg_q09", "chicken curry"],
    ["jg_q10", "chinese steamed buns"],
    ["jg_q11", "miso soup"],
    ["jg_q12", "Crispy Shrimp Tempura"],
    ["jg_q13", "Korean BBQ"],
    ["jg_q14", "Soybean Sprouts"],
    ["jg_q15", "Hot Dog Sauce"],
    ["jg_q16", "Vegan Pancakes"],
    ["jg_q17", "Classic Waffles"],
    ["jg_q18", "Russian Cabbage Soup"],
    ["jg_q19", "Beef Chop Suey"],
    ["jg_q20", "Salmon Rice Bowl"],
    ], columns=["qid", "query"])

In [ ]:
zy_queries = pd.DataFrame([
    ["zy_q00", "cheese mushroom pizza"], 
    ["zy_q01", "fruit mussie cake"], 
    ["zy_q02", "carrot cupcake"],
    ["zy_q03", "caramel banana"],
    ["zy_q04", "honey chocolate cupcake"],
    ["zy_q05", "fruit pie"],
    ["zy_q06", "banana ice cream"],
    ["zy_q07", "spicy instant noodles"],
    ["zy_q08", "spicy chicken sandwich"],
    ["zy_q09", "hot spicy pot"],
    ["zy_q10", "teriyaki beef"],
    ["zy_q11", "garlic bread"],
    ["zy_q12", "butter popcorn"],
    ["zy_q13", "apple pudding"],
    ["zy_q14", "pecan pie"],
    ["zy_q15", "oatmeal cookies"],
    ["zy_q16", "fish soup"],
    ["zy_q17", "gluten free pizza"],
    ["zy_q18", "chocolate cinnamon roll"],
    ["zy_q19", "chicken breast"]],
    columns=["qid", "query"])

In [ ]:
yn_queries = pd.DataFrame([["yn_q01", "teriyaki chicken"], ["yn_q02", "potato with green pepper and eggplants"], ["yn_q03", "roasted turkey"], ["yn_q04", "apple pie"], ["yn_q05", "coconut oil"],
                        ["yn_q06", "hot pot"], ["yn_q07", "spicy "], ["yn_q08", "curry chicken with potato"], ["yn_q09", "broccoli beef"], ["yn_q10", "vegetable salad with chicken"],
                        ["yn_q11", "mashed garlic"], ["yn_q12", "toast bread with jam"], ["yn_q13", "milk chocolate"], ["yn_q14", "coffee vanilla"], ["yn_q15", "fruit yogurt"],
                        ["yn_q16", "celery shrimp dumplings"], ["yn_q17", "french fries"], ["yn_q18", "salmon pasta"], ["yn_q19", "blueberry bagle"], ["yn_q20", "green bean with corn"]], columns=["qid", "query"])

In [ ]:
all_queries = pd.concat([jg_queries, zy_queries, yn_queries])

In [ ]:
qrels = pd.read_csv('/content/all_qrels.csv')

In [ ]:
qrels['label'].astype(np.int)

<ipython-input-12-2c3c688ed1a7>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  qrels['label'].astype(np.int)


0       3
1       4
2       3
3       3
4       3
       ..
3313    2
3314    2
3315    2
3316    2
3317    2
Name: label, Length: 3318, dtype: int64

In [ ]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(all_queries, test_size=30, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=10, random_state=SEED)

In [ ]:
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5
!pip install --upgrade python-terrier

In [ ]:
import onir_pt

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field="title")
br = pt.BatchRetrieve(recipe_index2)

In [ ]:
bm25 = pt.BatchRetrieve(recipe_index2, wmodel="BM25")
pipeline_bm25 = (bm25 % 100 >> pt.text.get_text(recipe_index2, ["title","ingredients","instructions"]) >> knrm)

pipeline_bm25.fit(
    train_topics,
    qrels,
    valid_topics,
    qrels)

[2022-12-16 04:43:16,273][onir_pt][DEBUG] [starting] validation
[2022-12-16 04:43:16,274][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:43:16,275][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/250 s<?, ?it/s]

[2022-12-16 04:43:16,591][onir_pt][DEBUG] [finished] batches: s] [250it] [793.99it/s]
[2022-12-16 04:43:16,610][onir_pt][DEBUG] [finished] validation s]
[2022-12-16 04:43:16,611][onir_pt][INFO] pre-validation: 0.1230
[2022-12-16 04:43:16,613][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:43:16,614][onir_pt][DEBUG] [starting] training
[2022-12-16 04:43:16,615][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-12-16 04:43:23,745][onir_pt][DEBUG] [finished] train pairs: [7.13s] [1024it] [143.64it/s]
[2022-12-16 04:43:23,747][onir_pt][DEBUG] [finished] training [7.13s]
[2022-12-16 04:43:23,748][onir_pt][INFO] training   it=0 loss=0.1864
[2022-12-16 04:43:23,748][onir_pt][DEBUG] [starting] validation
[2022-12-16 04:43:23,748][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:43:23,749][onir_pt][DEBUG] [starting] batches


...

[2022-12-16 04:49:48,259][onir_pt][DEBUG] [finished] batches: s] [250it] [490.69it/s]
[2022-12-16 04:49:48,288][onir_pt][DEBUG] [finished] validation s]
[2022-12-16 04:49:48,289][onir_pt][INFO] validation it=62 map=0.1261 ndcg=0.1464 P_10=0.1517
[2022-12-16 04:49:48,289][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:49:48,290][onir_pt][DEBUG] [starting] training
[2022-12-16 04:49:48,291][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2022-12-16 04:49:54,457][onir_pt][DEBUG] [finished] train pairs: [6.17s] [1024it] [166.10it/s]
[2022-12-16 04:49:54,460][onir_pt][DEBUG] [finished] training [6.17s]
[2022-12-16 04:49:54,464][onir_pt][INFO] training   it=63 loss=0.1662
[2022-12-16 04:49:54,464][onir_pt][DEBUG] [starting] validation
[2022-12-16 04:49:54,464][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:49:54,465][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/250 s<?, ?it/s]

[2022-12-16 04:49:54,962][onir_pt][DEBUG] [finished] batches: s] [250it] [503.40it/s]
[2022-12-16 04:49:54,990][onir_pt][DEBUG] [finished] validation s]
[2022-12-16 04:49:54,991][onir_pt][INFO] validation it=63 map=0.1268 ndcg=0.1471 P_10=0.1500
[2022-12-16 04:49:54,991][onir_pt][INFO] early stopping; model reverting back to it=43


In [ ]:
pt.Experiment(
    [bm25, pipeline_bm25],
    test_topics,
    qrels,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10", "mrt"],
    names=["bm25", "pipeline_bm25"]
)

[2022-12-16 04:50:06,950][onir_pt][DEBUG] using GPU (deterministic)
[2022-12-16 04:50:06,952][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/750 s<?, ?it/s]

[2022-12-16 04:50:08,599][onir_pt][DEBUG] [finished] batches: [1.65s] [750it] [455.66it/s]


,name,map,ndcg,ndcg_cut_10,P_10,mrt
0,bm25,0.903123,0.963844,0.941876,0.976667,190.667233
1,pipeline_bm25,0.734767,0.871773,0.786651,0.863333,249.966598
